In [ ]:
import pandas as pd

In [ ]:
filepath = "TetonLakesDataOrig.xlsx"
output_name = "TetonsLakesData.csv"
national_park = "Grand Tetons"

In [ ]:
# Read in Excel file
dictionary = pd.read_excel(filepath, sheet_name=None)

# Iterate through each DataFrame
for sheet_name, df in dictionary.items():
  # check if the first two columns are empty, then if they are drop them
  if df.iloc[:, 0].isnull().all() and df.iloc[:, 1].isnull().all():
    df = df.drop(df.columns[[0, 1]], axis=1)
    df = df.reset_index(drop=True)
  elif df.iloc[:, 0].isnull().all():
    df = df.drop(df.columns[[0]], axis=1)
    df = df.reset_index(drop=True)
  elif df.iloc[:, 1].isnull().all():
    df = df.drop(df.columns[[1]], axis=1)
    df = df.reset_index(drop=True)

  # Read down each column until the first value ignoring any that include the string "this table", then use that value as the column title.
  for column_index in range(df.shape[1]):
    for row_index in range(df.shape[0]):
      if not pd.isna(df.iloc[row_index, column_index]) and "this table" not in str(df.iloc[row_index, column_index]):
        df.columns.values[column_index] = df.iloc[row_index, column_index]
        # If the row below the first value is not blank add that value to the column title
        if not pd.isna(df.iloc[row_index+1, column_index]) and not df.iloc[row_index, column_index] == "Site":
          df.columns.values[column_index] = str(df.columns.values[column_index]) + " " + str(df.iloc[row_index+1, column_index])
        break
  # Filter the dataframe and for any cells that say no data replace that with a null value
  df = df.replace("no data", pd.NA)

  # Use the Location column in df to make a new column called Location_note which is anything after Inlake or Inlet, then remove any characters after the words Inlake or Inlet in the Location column
  df["Location"] = df["Location"].str.replace("inlet", "Inlet")
  df["Location_note"] = df["Location"].str.replace("Inlake", "").str.replace("Inlet", "").str.replace(" ", "").str.replace("Outlet", "")
  df["Location"] = df["Location"].astype(str).apply(lambda x: "Inlake" if "Inlake" in x else x)
  df["Location"] = df["Location"].astype(str).apply(lambda x: "Inlet" if "Inlet" in x else x)
  df["Location"] = df["Location"].astype(str).apply(lambda x: "Outlet" if "Outlet" in x else x)

  # Remove unnecessary columns from the dataframes
  # Iterate through the columns and find where there is only one value in the column or no values and drop those columns
  df_drop = []
  new_df_drop = []
  for column_index in range(df.shape[1]):
      if df.iloc[:, column_index].isnull().all() or df.iloc[:, column_index].notnull().sum() == 1:
          df_drop.append(column_index)
  # Drop the unnecessary columns
  df = df.drop(df.columns[df_drop], axis=1)

  # Remove all unecessary rows from the dataframes using the site column
  df = df[df["Site"].notna() & (df["Site"] != "Site")].reset_index(drop=True)

  # Take the Year and Month columns and make a new column called Date which has both of them in a datetime format
  df = df[df["Month"].notna() & (df["Month"] != "no data")].reset_index(drop=True)
  df["Date"] = pd.to_datetime(df["Year"].astype(str) + "-" + df["Month"].astype(str), format="%Y-%b")

  # Add new columns
  df["df_name"] = sheet_name
  df["Side"] = 2

  # Add df to the dictionary
  dictionary[sheet_name] = df

# Combine the dataframes and output a csv of the data
combine = pd.concat(dictionary.values(), ignore_index=True)
combine["National Park"] = national_park
combine.to_csv(output_name, index=False)
print(f"Data saved to the Colab file explorer as {output_name}")

ValueError: time data "1996a-Jun" doesn't match format "%Y-%b", at position 59. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.